# xml 안에서 데이터 추출
- 원하는 데이터들만 json type 으로 변경하고 그에 대한 분포를 dataframe 형태로 만들자
- 필요없는 이미지 리스트를 만들어 놓는게 좋을까 고민은 해볼 것(data root 경로를 뺀 상태에서 이미지 리스트를 파일로 저장해두자)

In [ ]:
XML_DIR = '/opt/ml/data/team_github/xml'
XML_LIST = [i for i in os.listdir(XML_DIR) if i.endswith('.xml')]
print(len(XML_LIST))
print(XML_LIST[0])

In [ ]:
all_info = []
from tqdm import tqdm
for x in tqdm(XML_LIST):
    all_info.extend(extract_xml_info(x))
print()
print(len(all_info))

In [ ]:
import pandas as pd

columns = ['root_folder', 'sub_folder', 'xml_name', 'file_name', 'occluded', 'classes', 'polygon', 'subclass']
target_list = ('crosswalk')

pandas_dict = pd.DataFrame(all_info, columns=columns)

pandas_dict.to_csv('crosswalk_temp.csv' ,header=True, index=False)

In [ ]:
target_frame.to_csv('XML_filtered(21_30).csv', header=True, index=False)